# Interactive Sunyaev-Zel'dovich Effect

This Jupyter notebook contains an interactive demo of the Sunyaev-Zel'dovich effect, an example of Inverse Compton Scattering. For a detailed discussion on the SZ effect, check out the page on AstroBaki (https://casper.ssl.berkeley.edu/astrobaki/index.php/SZ_Effect) as well as the paper by Carlstrom et al. 2002, ["Cosmology with the Sunyaev-Zel'dovich Effect"](https://www.annualreviews.org/doi/pdf/10.1146/annurev.astro.40.060401.093803).

In [1]:
%matplotlib inline 

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.ndimage
from scipy import ndimage

from bokeh.io import output_notebook
from bokeh.layouts import row, column, widgetbox, Spacer, layout
from bokeh.models import CustomJS, ColumnDataSource, Slider, ColorBar, LinearColorMapper
from bokeh.plotting import figure, output_file, show, ColumnDataSource

# Physical constants
m_e = 1e-27
m_p = 1836 * m_e
c = 3e10
k_B = 1.4e-16
h = 2 * np.pi * 1e-27
h_bar = 1e-27
steph_boltz = 6e-5
a_0 = 5e-9
e = 5e-10
eV = 1.6e-12

# Calculates Planck spectrum from frequency and temperature
def planck(nu, T):
    return 2 * h * nu ** 3 / (c ** 2 * (np.exp(h * nu / (k_B * T)) - 1))

In [2]:
# Constants specific to SZ effect scenario
T = 2.725
y = 1e-4

# Range of frequencies to plot
nu = np.linspace(20e9, 900e9, 10000)
dnu = nu[1] - nu[0]
def nu_index(my_nu):
    return int((my_nu - nu[0]) / dnu)

# Calculate SZ spectrum from formulae in Carlstrom et al., Annual Reviews of Astronomy Astrophysics vol 40, 2002
# Specifically, Equations 2, 3, and 4
x = h * nu / (k_B * T)
I_0 = 2 * (k_B * T) ** 3 / (h * c) ** 2
g = x ** 4 * np.exp(x) / (np.exp(x) - 1) ** 2 * (x * (np.exp(x) + 1) / (np.exp(x) - 1) - 4) * 1
dI = g * I_0 * y

In [3]:
# Set range for 1d plot of specific intensity difference
X = nu / 1e9
Y = dI / 1e-23 / 1e6

# Initial location of point on plot
pt_x = [X[0]]
pt_y = [Y[0]]

# Create column data source, which will change as slider is adjusted
pt_source = ColumnDataSource(data=dict(pt_x=pt_x, 
                                       pt_y=pt_y))


plot = figure(x_axis_type="log")
plot.line(X, 
          Y, 
          line_width=3, 
          line_alpha=0.6)
plot.line(X, 
          Y * 0, 
          line_width=3, 
          line_alpha=0.6,
          line_dash='dashed', 
          line_color='black')
plot.circle('pt_x', 
            'pt_y', 
            source=pt_source, 
            size=20, 
            color="red", 
            alpha=0.5)

plot.xaxis.axis_label = 'Frequency (GHz)'
plot.xaxis.axis_label_text_font_style = 'normal'
plot.yaxis.axis_label = 'Change in Specific Intensity (MJy/sr)'
plot.yaxis.axis_label_text_font_style = 'normal'
plot.title.text = 'Change in Specific Intensity vs. Frequency'

# Create dimensions for 2d visualization
N = 500
x = np.linspace(0, 1, N)
y = np.linspace(0, 1, N)
xx, yy = np.meshgrid(x, y)

# Add some pseudo CMB background using random noise + gaussian smoothing, for visual flair
d = ndimage.gaussian_filter(np.random.normal(scale=10, 
                                             size=(N,N)), 
                            sigma=25)

# Create a set of masks which will be our simulated clouds of free electrons.
# These will each cover an elliptical region in pixels for simplicity.
mask1 = (x[np.newaxis,:]-(200/500))**2 + (y[:,np.newaxis]-(300/500))**2 < (50/N)**2
mask2 = (x[np.newaxis,:]-(300/500))**2 + (y[:,np.newaxis]-(450/500))**2 < (40/N)**2
mask3 = (x[np.newaxis,:]-(400/500))**2 + (y[:,np.newaxis]-(100/500))**2 < (30/N)**2
mask4 = (x[np.newaxis,:]-(100/500))**2 + (y[:,np.newaxis]-(400/500))**2 < (10/N)**2
mask5 = (x[np.newaxis,:]-(100/500))**2/4 + (y[:,np.newaxis]-(50/500))**2 < (15/N)**2

region_indices = []
for i in range(N):
    for j in range(N):
        if any([mask1[i][j], mask2[i][j], mask3[i][j], mask4[i][j], mask5[i][j]]):
            region_indices.append((N * i + j, d[i,j]))
            d[i,j] += Y[0]

p = figure(x_range=(0, 1), y_range=(0, 1),
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])

# Controls z-axis limits of 2d graph
color_lim = 0.3
color_mapper = LinearColorMapper(palette="Viridis256", 
                                 low=-color_lim, 
                                 high=color_lim)

# Data source for image is array of pixel values, which will change as slider is adjusted
im_source = ColumnDataSource(data=dict(image=[d]))

p.image('image', 
        x=0, 
        y=0, 
        dw=1, 
        dh=1, 
        color_mapper=color_mapper, 
        source=im_source)

color_bar = ColorBar(color_mapper=color_mapper, 
                     location=(0, 0))
color_bar.title= 'MJy/sr'

p.add_layout(color_bar, 
             'right')
p.xaxis.axis_label = 'x'
p.yaxis.axis_label = 'y'
p.title.text = 'Simulated 2D Radio Map of the SZ Effect'

# Javascript callbacks that take changes in slider value and propagate them to all plots
callback = CustomJS(args=dict(pt_source=pt_source, 
                              im_source=im_source,
                              region_indices=region_indices,
                              X=X,
                              Y=Y,
                              N=500,
                              dnu=dnu,
                              nu0=nu[0]), code="""
    var pt_data = pt_source.data;
    var im_data = im_source.data;
    
    var image = im_data['image'];
    
    var pt_x = pt_data['pt_x']
    var pt_y = pt_data['pt_y']
    
    var nu_val = freq.value;
    var index = Math.round((nu_val - nu0) / dnu)
    pt_x[0] = X[index];
    pt_y[0] = Y[index];
    
    for (var i = 0; i < region_indices.length; i++) {
        image[0][region_indices[i][0]] = region_indices[i][1] + pt_y[0];
    }
    
    pt_source.change.emit();
    im_source.change.emit();
""")

# Create frequency slider element, initialized with frequency values
freq_slider = Slider(start=nu[0], 
                     end=nu[-1], 
                     value=nu[0], 
                     step=dnu,
                     title="Frequency Slider", 
                     show_value=False,
                     tooltips=True,
                     callback=callback)
callback.args["freq"] = freq_slider

# Sets the layout of items
l = layout([widgetbox(freq_slider), 
            [plot, p]], 
           sizing_mode='scale_width')

# Output to html file
# output_file("interactive_sz_effect.html", title="Interactive Sunyaev-Zeldovich Effect")

# Output within notebook
output_notebook()

show(l)

Loading BokehJS ...